## LoRA implementation with PyTorch

In [14]:
import torch
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [15]:
# Make torch deterministic
_ = torch.manual_seed(0)

### We will be training a network to classify MNIST digits and then fine-tune the network on a particular digit on which it doesn't perform well.

In [16]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', 
                                train=True, 
                                download=True, 
                                transform=transform)

# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, 
                                           batch_size=10, 
                                           shuffle=True)

# Load the MNIST test set
mnist_testset = datasets.MNIST(root='./data', 
                               train=False, 
                               download=True, 
                               transform=transform)

test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

# Define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Create the Neural Network to classify the digits, making it deep (more parameters) to better show the power of LoRA

In [42]:
# Create an deep neural network to classify MNIST digits

class NNet(nn.Module):
    def __init__(self, hidden_size_1=1000, hidden_size_2=2000):
        super(NNet,self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1) 
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2) 
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

net = NNet().to(device)

In [43]:
net

NNet(
  (linear1): Linear(in_features=784, out_features=1000, bias=True)
  (linear2): Linear(in_features=1000, out_features=2000, bias=True)
  (linear3): Linear(in_features=2000, out_features=10, bias=True)
  (relu): ReLU()
)

A scaled down representation of the neural network

<div style="width: 1200px;">Neural Network scaled down</div>
<center><img src="./assets/nn.svg" width="1200"></center>

### Set the optimizer and loss for our network training.

In [44]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

#### Training function

In [51]:
def train(train_loader, net, epochs=1, num_iters=None):
    iters = 0
    net.train()
    for epoch in range(epochs):
        total_loss = 0
        iterations = 0
        train_data = tqdm(train_loader, desc='Epoch {epoch}')
        for data in train_data:
            optimizer.zero_grad()
            iterations += 1
            X, y = data
            X, y = X.to(device), y.to(device)
            output = net(X.view(-1, 28*28))
            loss = loss_fn(output, y)
            total_loss += loss.item()
            running_loss = total_loss / iterations
            train_data.set_postfix(loss=running_loss)
            loss.backward()
            optimizer.step()
            

            break
                    
train(train_loader, net, epochs=1)

Epoch {epoch}:   0%|                                                            | 0/6000 [00:00<?, ?it/s, loss=3.31e-5]
